In [ ]:
import cv2 as cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time
import os



In [ ]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid2.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]
        
        slope_shoulder_avgs = []
        slope_shoulder_negatives_avgs = []
        slope_shoulder_positives_avgs = []
        slope_shoulder_positives = []
        slope_shoulder_negatives = []

        differenceShouldersList = []
        leftshoulderList = []
        rightshoulderList = []
        lsp=[]
        rsp=[]
        lhp=[]
        rhp=[]
        lefthipList = []
        righthipList = []
        midpointsholder_l=[]
        midpointhip_l=[]
        spinelength_l=[]
        ratio_l=[]
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    
                    
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))
                    slope_shoulder_l.append(slope_shoulder)
                    if slope_hip < 0:
                        slope_shoulder_negatives.append(slope_hip)
                    if slope_hip >= 0:
                        slope_shoulder_positives.append(slope_hip)
                    
                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
                    lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y])
                
                
                    slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)

                except:
                    
                    pass
                    

            

 
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                
    
    slope_shoulder_avgs.append(np.mean(slope_shoulder_l))
    slope_shoulder_negatives_avgs.append(np.mean(slope_shoulder_negatives))
    slope_shoulder_positives_avgs.append(np.mean(slope_shoulder_positives))
    
    slope_shoulder_l = []
    slope_shoulder_negatives = []
    slope_shoulder_positives = []
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid3.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]

        slope_shoulder_positives = []
        slope_shoulder_negatives = []

        differenceShouldersList = []
        leftshoulderList = []
        rightshoulderList = []
        lsp=[]
        rsp=[]
        lhp=[]
        rhp=[]
        lefthipList = []
        righthipList = []
        midpointsholder_l=[]
        midpointhip_l=[]
        spinelength_l=[]
        ratio_l=[]
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    
                    
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))
                    slope_shoulder_l.append(slope_shoulder)
                    if slope_hip < 0:
                        slope_shoulder_negatives.append(slope_hip)
                    if slope_hip >= 0:
                        slope_shoulder_positives.append(slope_hip)
                    
                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
                    lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y])
                
               
                
                    slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)
    

                except:
                    
                    pass
                    

            

 
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                
    slope_shoulder_avgs.append(np.mean(slope_shoulder_l))
    slope_shoulder_negatives_avgs.append(np.mean(slope_shoulder_negatives))
    slope_shoulder_positives_avgs.append(np.mean(slope_shoulder_positives))
    
    slope_shoulder_l = []
    slope_shoulder_negatives = []
    slope_shoulder_positives = []
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid4.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]

        slope_shoulder_positives = []
        slope_shoulder_negatives = []

        differenceShouldersList = []
        leftshoulderList = []
        rightshoulderList = []
        lsp=[]
        rsp=[]
        lhp=[]
        rhp=[]
        lefthipList = []
        righthipList = []
        midpointsholder_l=[]
        midpointhip_l=[]
        spinelength_l=[]
        ratio_l=[]
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    
                    
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))
                    
                    slope_shoulder_l.append(slope_shoulder)
                    if slope_hip < 0:
                        slope_shoulder_negatives.append(slope_hip)
                    if slope_hip >= 0:
                        slope_shoulder_positives.append(slope_hip)
                    
                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
                    lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y])
                
                    slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)

                except:
                    
                    pass
                    

            

 
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                
    print(len(slope_shoulder_l))
    slope_shoulder_avgs.append(np.mean(slope_shoulder_l))
    slope_shoulder_negatives_avgs.append(np.mean(slope_shoulder_negatives))
    slope_shoulder_positives_avgs.append(np.mean(slope_shoulder_positives))
    
    slope_shoulder_l = []
    slope_shoulder_negatives = []
    slope_shoulder_positives = []
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid5.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]

        slope_shoulder_positives = []
        slope_shoulder_negatives = []

        differenceShouldersList = []
        leftshoulderList = []
        rightshoulderList = []
        lsp=[]
        rsp=[]
        lhp=[]
        rhp=[]
        lefthipList = []
        righthipList = []
        midpointsholder_l=[]
        midpointhip_l=[]
        spinelength_l=[]
        ratio_l=[]
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    
                    
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))
                    
                    if slope_hip < 0:
                        slope_shoulder_negatives.append(slope_hip)
                    if slope_hip >= 0:
                        slope_shoulder_positives.append(slope_hip)
                    
                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
                    lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y])
                
                    slope_shoulder_l.append(slope_shoulder)
                
                    slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)

                except:
                    
                    pass
                    

            

 
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
               
    
    slope_shoulder_avgs.append(np.mean(slope_shoulder_l))
    slope_shoulder_negatives_avgs.append(np.mean(slope_shoulder_negatives))
    slope_shoulder_positives_avgs.append(np.mean(slope_shoulder_positives))
    
    slope_shoulder_l = []
    slope_shoulder_negatives = []
    slope_shoulder_positives = []
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid6.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]
        

        slope_shoulder_positives = []
        slope_shoulder_negatives = []

        differenceShouldersList = []
        leftshoulderList = []
        rightshoulderList = []
        lsp=[]
        rsp=[]
        lhp=[]
        rhp=[]
        lefthipList = []
        righthipList = []
        midpointsholder_l=[]
        midpointhip_l=[]
        spinelength_l=[]
        ratio_l=[]
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    
                    
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))
                    
                    if slope_hip < 0:
                        slope_shoulder_negatives.append(slope_hip)
                    if slope_hip >= 0:
                        slope_shoulder_positives.append(slope_hip)
                    
                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
                    lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y])
                
                    slope_shoulder_l.append(slope_shoulder)
                
                    slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)

                except:
                    
                    pass
                    

            

 
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
               
    
    slope_shoulder_avgs.append(np.mean(slope_shoulder_l))
    slope_shoulder_negatives_avgs.append(np.mean(slope_shoulder_negatives))
    slope_shoulder_positives_avgs.append(np.mean(slope_shoulder_positives))
    
    slope_shoulder_l = []
    slope_shoulder_negatives = []
    slope_shoulder_positives = []
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid7.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]
        

        slope_shoulder_positives = []
        slope_shoulder_negatives = []

        differenceShouldersList = []
        leftshoulderList = []
        rightshoulderList = []
        lsp=[]
        rsp=[]
        lhp=[]
        rhp=[]
        lefthipList = []
        righthipList = []
        midpointsholder_l=[]
        midpointhip_l=[]
        spinelength_l=[]
        ratio_l=[]
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    
                    
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))
                    
                    if slope_hip < 0:
                        slope_shoulder_negatives.append(slope_hip)
                    if slope_hip >= 0:
                        slope_shoulder_positives.append(slope_hip)
                    
                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
                    lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y])
                
                    slope_shoulder_l.append(slope_shoulder)
                
                    slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)

                except:
                    
                    pass
                    

            

 
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
               
    
    slope_shoulder_avgs.append(np.mean(slope_shoulder_l))
    slope_shoulder_negatives_avgs.append(np.mean(slope_shoulder_negatives))
    slope_shoulder_positives_avgs.append(np.mean(slope_shoulder_positives))
    
    slope_shoulder_l = []
    slope_shoulder_negatives = []
    slope_shoulder_positives = []
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid8.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]

        slope_shoulder_positives = []
        slope_shoulder_negatives = []

        differenceShouldersList = []
        leftshoulderList = []
        rightshoulderList = []
        lsp=[]
        rsp=[]
        lhp=[]
        rhp=[]
        lefthipList = []
        righthipList = []
        midpointsholder_l=[]
        midpointhip_l=[]
        spinelength_l=[]
        ratio_l=[]
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    
                    
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))
                    
                    if slope_hip < 0:
                        slope_shoulder_negatives.append(slope_hip)
                    if slope_hip >= 0:
                        slope_shoulder_positives.append(slope_hip)
                    
                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
                    lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y])
                
                    slope_shoulder_l.append(slope_shoulder)
                
                    slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)

                except:
                    
                    pass
                    

            

 
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
               
    
    slope_shoulder_avgs.append(np.mean(slope_shoulder_l))
    slope_shoulder_negatives_avgs.append(np.mean(slope_shoulder_negatives))
    slope_shoulder_positives_avgs.append(np.mean(slope_shoulder_positives))
    
    slope_shoulder_l = []
    slope_shoulder_negatives = []
    slope_shoulder_positives = []
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid9.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]
        

        slope_shoulder_positives = []
        slope_shoulder_negatives = []

        differenceShouldersList = []
        leftshoulderList = []
        rightshoulderList = []
        lsp=[]
        rsp=[]
        lhp=[]
        rhp=[]
        lefthipList = []
        righthipList = []
        midpointsholder_l=[]
        midpointhip_l=[]
        spinelength_l=[]
        ratio_l=[]
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    
                    
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))
                    
                    if slope_hip < 0:
                        slope_shoulder_negatives.append(slope_hip)
                    if slope_hip >= 0:
                        slope_shoulder_positives.append(slope_hip)
                    
                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
                    lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y])
                
                    slope_shoulder_l.append(slope_shoulder)
                
                    slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)

                except:
                    
                    pass
                    

            

 
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
               
    
    slope_shoulder_avgs.append(np.mean(slope_shoulder_l))
    slope_shoulder_negatives_avgs.append(np.mean(slope_shoulder_negatives))
    slope_shoulder_positives_avgs.append(np.mean(slope_shoulder_positives))
    
    slope_shoulder_l = []
    slope_shoulder_negatives = []
    slope_shoulder_positives = []
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid10.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]
        

        slope_shoulder_positives = []
        slope_shoulder_negatives = []

        differenceShouldersList = []
        leftshoulderList = []
        rightshoulderList = []
        lsp=[]
        rsp=[]
        lhp=[]
        rhp=[]
        lefthipList = []
        righthipList = []
        midpointsholder_l=[]
        midpointhip_l=[]
        spinelength_l=[]
        ratio_l=[]
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    
                    
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))
                    
                    if slope_hip < 0:
                        slope_shoulder_negatives.append(slope_hip)
                    if slope_hip >= 0:
                        slope_shoulder_positives.append(slope_hip)
                    
                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
                    lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y])
                
                    slope_shoulder_l.append(slope_shoulder)
                
                    slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)

                except:
                    
                    pass
                    

            

 
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
               
    
    slope_shoulder_avgs.append(np.mean(slope_shoulder_l))
    slope_shoulder_negatives_avgs.append(np.mean(slope_shoulder_negatives))
    slope_shoulder_positives_avgs.append(np.mean(slope_shoulder_positives))
    
    slope_shoulder_l = []
    slope_shoulder_negatives = []
    slope_shoulder_positives = []
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid11.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]
        
        slope_shoulder_positives = []
        slope_shoulder_negatives = []

        differenceShouldersList = []
        leftshoulderList = []
        rightshoulderList = []
        lsp=[]
        rsp=[]
        lhp=[]
        rhp=[]
        lefthipList = []
        righthipList = []
        midpointsholder_l=[]
        midpointhip_l=[]
        spinelength_l=[]
        ratio_l=[]
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    
                    
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))
                    
                    if slope_hip < 0:
                        slope_shoulder_negatives.append(slope_hip)
                    if slope_hip >= 0:
                        slope_shoulder_positives.append(slope_hip)
                    
                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
                    lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y])
                
                    slope_shoulder_l.append(slope_shoulder)
                
                    slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)

                except:
                    
                    pass
                    

            

 
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
               
    
    slope_shoulder_avgs.append(np.mean(slope_shoulder_l))
    slope_shoulder_negatives_avgs.append(np.mean(slope_shoulder_negatives))
    slope_shoulder_positives_avgs.append(np.mean(slope_shoulder_positives))
    
    slope_shoulder_l = []
    slope_shoulder_negatives = []
    slope_shoulder_positives = []
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [ ]:
data={'slope_shoulder_averages': slope_shoulder_avgs,
      'slope_shoulder_negatives_avgs': slope_shoulder_negatives_avgs,
      'slope_shoulder_positives_avgs': slope_shoulder_positives_avgs
      }
dataF=pd.DataFrame(data)


In [ ]:
dataF

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
actions = [
    'optimal_shoulderfluctuation',
]

In [ ]:
labels = ["slope_shoulder"]

In [ ]:
data = slope_shoulder_l

In [ ]:
data